In [1]:
!nvidia-smi

Mon Nov 16 15:25:38 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
os.chdir('/content/drive/My Drive/GitHub/mitochondria_segmentation/src')
print(os.getcwd())
#print(os.listdir())

/content/drive/My Drive/GitHub/mitochondria_segmentation/src


In [3]:
!python train.py

Data loaded.
device: cuda:0
train.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs, dtype=torch.float32)
train.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  lables = torch.tensor(lables, dtype=torch.long)
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
batch.
bat